### 1 - Imports

In [ ]:
import torch
import os
import time
import torch.nn as nn
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import utils as u
from torchvision.transforms import Compose
from IPython.core.display import HTML
import json
import math

import utils

%matplotlib inline

In [ ]:
print(torch.__version__)

In [ ]:
with open('../input/configs/configs.json') as data_file:
    config = json.load(data_file)

In [ ]:
curr_folder = 'out'
if not os.path.exists(curr_folder):
    os.makedirs(curr_folder)

In [ ]:
batch_size = 84
steps_before_print = 300
num_workers = 0
step_size = 2
num_frames = 32 // step_size

### 2 - Seting up Data Loaders

In [ ]:
std, mean = [0.2674, 0.2676, 0.2648], [0.4377, 0.4047, 0.3925]

In [ ]:
transform = Compose([
    u.GroupResize((100, 160)),
    u.GroupRandomCrop((140, 100)),
    u.GroupRandomRotation(18),
    u.GroupCenterCrop((96, 96)),
    u.GroupToTensor(),
    u.GroupNormalize(std=std, given_mean=mean),
])

In [ ]:
transform_validation = Compose([
    u.GroupResize((100, 160)),
    u.GroupCenterCrop((96, 96)),
    u.GroupToTensor(),
    u.GroupNormalize(std=std, given_mean=mean),
])

In [ ]:
train_data = u.VideoFolder(
    root=config['train_data_folder'],
    csv_file_input=config['full_train_data_csv'],
    csv_file_labels=config['full_labels_csv'],
    clip_size=num_frames,
    nclips=1,
    step_size=step_size,
    is_val=False,
    transform=transform,
)

In [ ]:
train_loader = torch.utils.data.DataLoader(
    train_data,
    batch_size=batch_size,
    shuffle=True,
    num_workers=num_workers,
    pin_memory=False,
    drop_last=True)

In [ ]:
validation_data = u.VideoFolder(
    root=config['validation_data_folder'],
    csv_file_input=config['full_validation_data_csv'],
    csv_file_labels=config['full_labels_csv'],
    clip_size=num_frames,
    nclips=1,
    step_size=step_size,
    is_val=False,
    transform=transform_validation,
)

In [ ]:
validation_loader = torch.utils.data.DataLoader(
    validation_data,
    batch_size=batch_size,
    shuffle=True,
    num_workers=num_workers,
    pin_memory=False,
    drop_last=True
)

In [ ]:
test_data = u.VideoFolder(
    root=config['test_data_folder'],
    csv_file_input=config['full_test_data_csv'],
    csv_file_labels=config['full_labels_csv'],
    clip_size=num_frames,
    nclips=1,
    step_size=step_size,
    is_val=False,
    transform=transform_validation,
)

In [ ]:
test_loader = torch.utils.data.DataLoader(
    validation_data,
    batch_size=batch_size,
    shuffle=True,
    num_workers=num_workers,
    pin_memory=False,
    drop_last=True
)

In [ ]:
def save_model(model, use_ts=False):
    if use_ts:
        time_stamp = time.strftime("%d_%b_%Y_%Hh%Mm", time.gmtime())
        torch.save(model.state_dict(), curr_folder + '/{}.h5'.format(time_stamp))
    else:
        torch.save(model.state_dict(), curr_folder + '/{}.h5'.format('best_model'))

### 3 - Model definition

In [ ]:
model = u.CombinedModel(batch_size=batch_size)

In [ ]:
print(model)

In [ ]:
file = 'model_40_10.ckp'
if file != '':
    print('Model LOADED: ', '../input/configs' + '/' + file)
    loaded_dict = torch.load('../input/configs' + '/' + file, map_location=torch.device('cpu'))
    model.load_state_dict(loaded_dict)
else:
    print('No model loaded.')

In [ ]:
criterion = nn.CrossEntropyLoss()

In [ ]:
ct = 0
for param in model.parameters():
    if param.requires_grad:
        ct += 1
print(ct)

In [ ]:
for param in model.fm.parameters():
    param.requires_grad = False

In [ ]:
def train(epochs):
    print("Training is about to start...")
    best_valdiation_loss = model.best_validation_loss

    for epoch in range(epochs):
        step = 0
        epoch_loss = 0
        epoch_acc = 0
        times_calculated = 0
        total_size = len(train_loader)
        for i, (images, labels) in enumerate(train_loader):
            model.train()

            if torch.cuda.is_available():
                images = images.cuda()
                labels = labels.cuda()

            optimizer.zero_grad()
            outputs = model(images)
            loss = criterion(outputs.squeeze(), labels)
            loss.backward()
            optimizer.step()

            step += 1
            epoch_loss += loss.item()
            if step % 50 == 0:
                print(f'step {step} of {total_size}')

            if step % steps_before_print == 0:
                model.eval()
                validation_loss, accuracy = utils.calculate_loss_and_accuracy(validation_loader, model, criterion,
                                                                              stop_at=1200)
                epoch_acc += accuracy
                times_calculated += 1
                print(f'Iteration: {step}/{total_size} - ({step * 100 / total_size:.2f}%). Loss: {loss.item()}. Accuracy: {accuracy}')
                print(f'val_loss: {validation_loss}, best val loss: {model.best_validation_loss}')
                if validation_loss < model.best_validation_loss:
                    model.best_validation_loss = validation_loss
                    print('Saving best model')
                    save_model(model)
                del validation_loss
            del loss, outputs, images, labels

        model.epochs += 1

        print('Epoch({}) avg loss: {} avg acc: {}'.format(epoch, epoch_loss/step, epoch_acc/times_calculated))
        

In [ ]:
ct = 0
for param in model.parameters():
    if param.requires_grad:
        ct += 1
print(ct)

In [ ]:
if torch.cuda.is_available():
    print('Cuda is available!')
    model.cuda()

In [ ]:
learning_rate = 0.01
optimizer = torch.optim.SGD(filter(lambda p: p.requires_grad, model.parameters()), lr=learning_rate, momentum=0.9)
train(40)
save_model(model, use_ts=True)

In [ ]:
learning_rate = learning_rate / 10
optimizer = torch.optim.SGD(filter(lambda p: p.requires_grad, model.parameters()), lr=learning_rate, momentum=0.9)
train(10)
save_model(model, use_ts=True)

In [ ]:
print(len(train_loader), len(validation_loader), len(test_loader))

### 4 - Testing the models

In [ ]:
model.eval()
validation_loss, accuracy = u.calculate_loss_and_accuracy(validation_loader, model, criterion, 83)
train_loss, train_accuracy = u.calculate_loss_and_accuracy(train_loader, model, criterion, 600)
test_loss, test_accuracy = u.calculate_loss_and_accuracy(test_loader, model, criterion, 83)
print(f'Accuracy-- Validation: {accuracy} \t Train: {train_accuracy} \t Test: {test_accuracy}')
print(f'Loss-- Validation: {validation_loss} \t Train: {train_loss} \t Test: {test_loss}')